# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Основные пункты нашего исследования:
* Открытие данных
* Предобработка данных
* Обработка пропущенных значений
* Обработка дубликатов
* Проведение лемматизации
* Категорирование данных
* Ответы на вопросы исследования
* Проведение общего выводы проекта

Входные данные:
    Название таблицы - `table`
    
Столбцы:

* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита
* dob_years_rules - тип занятости в соответствии с возрастом
* goals - обобщенные цели получения кредита

## Шаг 1. Откроем файл с данными и изучаем общую информацию

In [1]:
# импортируем библиотеки
import pandas as pd
from pymystem3 import Mystem

In [2]:
# создаем название нашей таблице 'table'
table = pd.read_csv('/datasets/data.csv')

In [3]:
# вызываем 15 первых строк таблицы
table.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

Взглянув на таблицу, мы видим пропуски NaN в некоторых столбцах - `days_employed` и `total_income`.   Возможно, это утеря данных. В любом случае, оставлять в таком виде таблицу нельзя. Заменять на нули эти значения тоже не имеет смысла - банк бы не дал кредит заемщику без трудового стажа и с нулевым доходом. Данные пропущенные значения лучше всего заполнить средним значением соответствующих заемщиков. Для этого следует написать функцию, которая определяет тип занятости в зависимости от возраста и рассчитать медиану трудового стажа в соответствии этой занятости. Этим значением и заполнить пропуски в столбце `days_employed`. Подобным образом поступим и со столбцом `total_income`, посчитав медиану общего заработка исходя из их занятости.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [4]:
# запрашиваем информацию о таблице
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [5]:
# выяснем суммарное количество пропущенных значений
table.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Мы видим пропущенные значения в столбце `days_employed` и `total_income`. Количество пропущенных значений в столбцах равно друг другу и равно 2174. Возможно, эти данные были записаны некорректно. 


Напишем функцию `dob_years_rules` которая фильтрует наших заемщиков,исходя от их возраста, на 3 категории - `учаться`, `работают` и `на пенсии`. Затем добавим результаты нашей функции в отдельный столбец под названием `dob_years_rules`.


После этого рассчитаем медиану для каждой категории исходя из рабочих дней,а,затем, из заработанной суммы. Полученными значениями заполняем пропущенные значения NaN в столбцах `days_employed` и `total_income` соответственно.


In [6]:
# пишем функцию dob_years_rules()
def dob_years_rules(age):
    if age < 22:
        return 'учаться'
    if (22 < age) and (age < 65):
        return 'работают'
    else:
        return 'на пенсии'
table['dob_years_rules'] = table ['dob_years'].apply(dob_years_rules)

In [7]:
# считаем медиану трудового стажа у учащихся исходя из столбца 'dob_years_rules' и 'days_employed'
student_median = table.loc[table.loc[:,'dob_years_rules'] == 'учаться']['days_employed'].median()
print('Медиана общего трудового стажа у учащихся:',student_median)

Медиана общего трудового стажа у учащихся: -757.6126236444821


In [8]:
#считаем медиану трудового стажа у работающих исходя из столбца 'dob_years_rules' и 'days_employed'
adult_median = table.loc[table.loc[:,'dob_years_rules'] == 'работают']['days_employed'].median()
print('Медиана общего трудового стажа у работающих:',adult_median)

Медиана общего трудового стажа у работающих: -1305.553988687328


In [9]:
# считаем медиану трудового стажа у пенсионеров исходя из столбца 'dob_years_rules' и 'days_employed'
senior_median = table.loc[table.loc[:,'dob_years_rules'] == 'на пенсии']['days_employed'].median()
print('Медиана общего трудового стажа у пенсионеров:',senior_median)

Медиана общего трудового стажа у пенсионеров: 351421.50795311574


In [11]:
#заполняем пропуски в столбце 'days_employed' у заемщиков,
#которые учаться('dob_years_rules' = учаться) значением медианы 
table.loc[(table['dob_years_rules'] == 'учаться')&
          (table['days_employed'].isna()), 'days_employed'] = student_median

In [12]:
#заполняем пропуски в столбце 'days_employed' у заемщиков,
#которые работают('dob_years_rules' = работают) значением медианы 
table.loc[(table['dob_years_rules'] == 'работают')&
          (table['days_employed'].isna()), 'days_employed'] = adult_median

In [13]:
#заполняем пропуски в столбце 'days_employed' у заемщиков,
#которые находятся на пенсии('dob_years_rules' = на пенсии) значением медианы
table.loc[(table['dob_years_rules'] == 'на пенсии')&
          (table['days_employed'].isna()), 'days_employed'] = senior_median

In [14]:
# считаем медиану заработка у учащихся исходя из столбца 'dob_years_rules' и 'total_income'
student_median_2 = table.loc[table.loc[:,'dob_years_rules'] == 'учаться']['total_income'].median()
print('Медиана общего заработка у учащихся:',student_median_2)

Медиана общего заработка у учащихся: 128265.72087100228


In [15]:
# считаем медиану заработка у работающих исходя из столбца 'dob_years_rules' и 'total_income'
adult_median_2 = table.loc[table.loc[:,'dob_years_rules'] == 'работают']['total_income'].median()
print('Медиана общего заработка у работающих:',adult_median_2)

Медиана общего заработка у работающих: 146651.1734759125


In [16]:
# считаем медиану заработка у пенсионеров исходя из столбца 'dob_years_rules' и 'total_income'
seniors_median_2 = table.loc[table.loc[:,'dob_years_rules'] == 'на пенсии']['total_income'].median()
print('Медиана общего заработка у пенсионеров:',seniors_median_2)

Медиана общего заработка у пенсионеров: 116612.71539590893


In [17]:
# заполняем пропуски в столбце 'total_income' у заемщиков,
# которые учаться('dob_years_rules' = учаться) значением медианы 
table.loc[(table['dob_years_rules'] == 'учаться')&
          (table['total_income'].isna()), 'total_income'] = student_median_2

In [18]:
#заполняем пропуски в столбце 'total_income' у заемщиков,
#которые работают('dob_years_rules' = работают) значением медианы 
table.loc[(table['dob_years_rules'] == 'работают')&
          (table['total_income'].isna()), 'total_income'] = adult_median_2

In [19]:
#заполняем пропуски в столбце 'total_income' у заемщиков,
#которые работают('dob_years_rules' = работают) значением медианы 
table.loc[(table['dob_years_rules'] == 'на пенсии')&
          (table['total_income'].isna()), 'total_income'] = seniors_median_2

Еще раз проверим общую информацию о таблице. Исходя из данных, в таблице теперь нет пропущенных значений.

**Вывод**

Пропуски, которые были в двух столбцах `days_employed` и `total_income` было заполнены соответствующими данными путем нахождения среднего значения у разных категорий заемщиков исходя из их возраста.

### Замена типа данных

В таблице представлено два столбца с вещественными типами данных - `days_employed` и `total_income`. Поскольку дни не могут быть вещественными числами, переведем их в целые, используя метод astype('int').


In [20]:
# заменяем вещественный тип данных(float) в столбце 'days_employed' на целочисленный(int)
# путем использовании функции astype()
table['days_employed'] = table['days_employed'].astype('int')

Проверим общую информацию о таблице.

In [21]:
#запрашиваем информацию о таблице
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  int64  
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
 12  dob_years_rules   21525 non-null  object 
dtypes: float64(1), int64(6), object(6)
memory usage: 2.1+ MB


Данные в столбце `days_employed` заменены на целочисленные.

**Вывод**

Перевод вещественных данных в целочисленные был сделан при помощи метода astype(), который может изменить наше значение в любой тип данных (в отличии от функции to_numeric(), который превращает значение только в вещественное значение(float), что нам не подходит)

### Обработка дубликатов

При обработке дубликатов наперво посмотрим на количество абсолютных повторения при помощи метода duplicated().sum(). Затем, удалим идентичные строки, приминив метод drop_duplicates()

In [22]:
# проверяем данные таблицы на наличие прямых дубликатов
table.duplicated().sum()

54

In [23]:
# избавляемся от данных строк-дубликатив при помощи метода drop_duplicates()
# при помощи метода reset_index(drop=True) изменяем индексы на новые и удаляем старые
table = table.drop_duplicates().reset_index(drop=True)

In [24]:
# проверяем еще раз количество повторов
table.duplicated().sum()

0

Проверив количество идентичных повторов, проверим, есть и в столбцах уникальные значения при помощи метода unique()

In [25]:
# проверим, есть ли уникальные значение в столбце 'education' с помощью метода unique()
table['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Как видим, в столбце присутствуют значения, написанные в верхнем регистре. Приведем все значения к нижнему регистру путем использования метода str.lower().

In [26]:
# выполним перевод в нижний регистр всех значений в столбце 'education' и
# сохраняем новое значение в столбце.
table['education'] = table['education'].str.lower()

In [27]:
# проверяем, остались ли еще необычные значения в столбце 'education' при помощи unique()
table['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

В столбце с образованием нет больше необычных значений. Проверим тем же способом остальные столбцы таблицы на наличие уникальных значений.

In [28]:
# проверим, есть ли уникальные значение в столбце 'family_status' при помощи unique()
table['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [29]:
# выполним перевод в нижний регистр всех значений в столбце 'family_status' и сохраняем новое значение в столбце.
table['family_status'] = table['family_status'].str.lower()

In [30]:
# проверяем, остались ли еще необычные значения в столбце 'family_status' при помощи unique()
table['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

In [31]:
# проверим, есть ли уникальные значение в столбце 'income_type' при помощи unique()
# похоже, что их нет
table['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [32]:
# проверим, есть ли уникальные значение в столбце 'purpose' при помощи unique()
# похоже, что их нет
table['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Вывод**

Было выполнено удаление дубликатов как прямых(методом drop_duplicates()), так и уникальных значений (методом unique()) путем приведением слов к нижнему регистру при помощи метода str.lower().

### Лемматизация

Более наглядно изучить цели получения кредита нам поможет лемматизация - приведение слова к его словарной форме. При помощи метода value_counts посмотрим на значения столбца `purpose` который возвращает уникальные значения и количество их упоминаний. Далее вызовем лемматизацию к столбце, используя apply(), который берёт значения столбца и применяет к нему функцию из своего аргумента(в данном случае аргумент - m.lemmatize). Выделив основные леммы, напишем функцию. Затем создадим столбец под названием `goals`, в котором будут записываться результаты нашей функции.

In [33]:
# вызываем лемматизацию 
m = Mystem() 

In [34]:
# вызываем метод value_counts() для подсчета уникальных значений
table['purpose'].value_counts()

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
покупка жилья для сдачи                   652
операции с жильем                         652
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [35]:
# вызываем метод apply(), чтоб применить к столбцу лемитизацию
table['purpose'] = table['purpose'].apply(m.lemmatize)

In [57]:
# выводим уникальные значения
table['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Пишем функцию, которая разделяет цели, на которые берется кредит, на 4 категории - 'Недвижимость','Автомобиль','Образование', 'Свадьба'(исходя из результатов, которые мы увидели после применения метода value_counts()). Затем добавляем эти значение в новый столбец `goals`.

In [37]:
# пишем функцию purpose_reason()
def purpose_reason(purpose):
    if ('жилье' in purpose) or ('недвижимость' in purpose):
        return 'Недвижимость'
    if 'автомобиль' in purpose:
        return 'Автомобиль'
    if 'образование' in purpose:
        return 'Образование'
    if 'свадьба' in purpose:
        return 'Свадьба'
table['goals'] = table['purpose'].apply(purpose_reason)   

In [38]:
# подсчитываем количество людей в зависимости от цели займа кредита
print(table['goals'].value_counts())

Недвижимость    3883
Образование     2699
Автомобиль       973
Свадьба          793
Name: goals, dtype: int64


**Вывод**

Применив лематизацию выяснилось, что большая часть заемщиков вкладывается в недвижимость.

### Категоризация данных

В нашей таблице есть два критерия - образования и семейный статус, которые можно категоризировать. Объединим эти данные в группу по этим критериям. Для этого создадим новую таблицу `education_dict`. Из таблицы table перенесем столбцы `education` и `education_id`.Затем удалим из новой таблицы дубликаты и формируем ее в порядке возрастания столбца `education_id` при помощи метода sort_values(). После создадим таблицу `family_dict` по такому же принципу.

In [39]:
# создаем таблицу education_dict
# удаляем дубликаты методом drop_duplicates(), фармируем по увеличению столбца 'education_id'
education_dict = pd.DataFrame(data = table, columns = ['education', 'education_id'])
display(education_dict.drop_duplicates().reset_index(drop=True).sort_values('education_id'))

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [40]:
# создаем таблицу family_dict
# удаляем дубликаты методом drop_duplicates(), формируем по увеличению столбца 'family_status_id'
family_dict = pd.DataFrame(data = table, columns = ['family_status', 'family_status_id'])
display(family_dict.drop_duplicates().reset_index(drop=True).sort_values('family_status_id'))

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


**Вывод**

Мы выделили два словаря для категоризации, касающиеся образования и семейного статуса.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для того, чтоб узнать завиимость между наличием детей и возвратом кредита в срок мы найдем процент должников в каждой группе(у которых нет детей, один. два ребенка и др.). Эти данные наиболее наглядно и удобно можно отразить в сводной таблице, используя метод pivot_table(). Новую таблицу назовем `children_pivot`.

In [41]:
# создаем сводную таблицу методом pivot_table()
children_pivot = table.pivot_table(index = ['children'], values ='debt', aggfunc=('count', 'sum'))

In [42]:
# переименовываем название столбцов для большей наглядности при помощи метода set_axis()
children_pivot.set_axis(['Колличество заемщиков','Колличество должников'],axis='columns',inplace = True)

In [43]:
# находим процентое соотношение количества должников к общему колличеству заемщиков
# сохраняем эти значения в столбце 'Процентное соотношение'
children_pivot['Процентное соотношение'] = children_pivot['Колличество должников'] / children_pivot['Колличество заемщиков'] * 100

In [44]:
# сортируем нашу сводную таблицу по убыванию по столбцу 'Процентное соотношение' при помощи метода sort_values()
children_pivot.sort_values(by='Процентное соотношение', ascending=False)

,Колличество заемщиков,Колличество должников,Процентное соотношение
children,,,
20,76,8,10.526316
4,41,4,9.756098
2,2052,194,9.454191
1,4809,444,9.232689
3,330,27,8.181818
0,14107,1063,7.535266
-1,47,1,2.127660
5,9,0,0.000000


**Вывод**

Как видим, колличество детей определенно влияет на возврат кредита в срок. Наибольшее колличество должников имеют 20 детей. Бездетные заемщики состаавляют лишь 7 процентов от общего числа должников.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для того, чтоб узнать завиимость между семейным положением и возвратом кредита в срок мы найдем процент должников в каждой группе(не женат / не замужем, гражданский брак и др.). Эти данные наиболее наглядно и удобно можно отразить в сводной таблице, используя метод pivot_table(). Новую таблицу назовем family_pivot().

In [45]:
# создаем сводную таблицу методом pivot_table()
family_pivot = table.pivot_table(index = ['family_status'], values ='debt', aggfunc=('count', 'sum'))

In [46]:
# переименовываем название столбцов для большей наглядности при помощи метода set_axis()
family_pivot.set_axis(['Колличество заемщиков','Колличество должников'],axis='columns',inplace = True)

In [47]:
# находим процентое соотношение количества должников к общему колличеству заемщиков
# сохраняем эти значения в столбце 'Процентное соотношение'
family_pivot['Процентное соотношение'] = family_pivot['Колличество должников']/ family_pivot['Колличество заемщиков'] * 100

In [48]:
# сортируем нашу сводную таблицу по убыванию по столбцу 'Процентное соотношение' при помощи метода sort_values()
family_pivot.sort_values(by='Процентное соотношение', ascending=False)

,Колличество заемщиков,Колличество должников,Процентное соотношение
family_status,,,
не женат / не замужем,2810,274,9.750890
гражданский брак,4163,388,9.320202
женат / замужем,12344,931,7.542126
в разводе,1195,85,7.112971
вдовец / вдова,959,63,6.569343


**Вывод**

Как ни странно, наибольшее количесто должников - это свободные люди, не состоящие в официальных отношениях -  почти 10 %. На втором месте с небольшим отрывом идут люди, состоящие в гражданском браке(около 10 %). Самые исправные плательщики в этой категори - вдовы и вдовцы, которые состоаляют 6.6 %.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для того, чтоб узнать зависимость между уровнем дохода и возвратом кредита в срок мы найдем процент должников в каждой группе(учаться, работают, на пенсии). Эти данные наиболее наглядно и удобно можно отразить в сводной таблице, используя метод pivot_table(). Новую таблицу назовем dedt_pivot().

In [49]:
# создаем сводную таблицу методом pivot_table()
dedt_pivot = table.pivot_table(index = ['dob_years_rules'], values ='debt', aggfunc=('count', 'sum'))

In [50]:
# переименовываем название столбцов для большей наглядности при помощи метода set_axis()
dedt_pivot.set_axis(['Колличество заемщиков','Колличество должников'],axis='columns',inplace = True)

In [51]:
# находим процентое соотношение количества должников к общему колличеству заемщиков
# сохраняем эти значения в столбце 'Процентное соотношение'
dedt_pivot['Процентное соотношение'] = dedt_pivot['Колличество должников']/ dedt_pivot['Колличество заемщиков'] * 100

In [52]:
# сортируем нашу сводную таблицу по убыванию по столбцу 'Процентное соотношение' при помощи метода sort_values()
dedt_pivot.sort_values(by='Процентное соотношение', ascending=False)

,Колличество заемщиков,Колличество должников,Процентное соотношение
dob_years_rules,,,
учаться,277,27,9.747292
работают,20113,1640,8.153930
на пенсии,1081,74,6.845513


**Вывод**

Студенты, которые заняты учебой и работой, являются наибольшими должниками в банке - 10 %. Люди, которые отдают себя только работе и не грызут гранит науки, идут следом и сотавляют около 8 %. Среди лиц пожилого возраста, находящихся на заслуженном отпуске, меньше всего должников - около 7 %.

- Как разные цели кредита влияют на его возврат в срок?

Для того, чтоб узнать зависимость между целью займа и возвратом кредита в срок мы найдем процент должников в каждой группе(покупка авто, образование, свадьба или недвижимость). Эти данные наиболее наглядно и удобно можно отразить в сводной таблице, используя метод pivot_table(). Новую таблицу назовем `goals_pivot`.

In [53]:
# создаем сводную таблицу методом pivot_table()
goals_pivot = table.pivot_table(index = ['goals'], values ='debt', aggfunc=('count', 'sum'))

In [54]:
# переименовываем название столбцов для большей наглядности при помощи метода set_axis()
goals_pivot.set_axis(['Колличество заемщиков','Колличество должников'],axis='columns',inplace = True)

In [55]:
# находим процентое соотношение количества должников к общему колличеству заемщиков
# сохраняем эти значения в столбце 'Процентное соотношение'
goals_pivot['Процентное соотношение'] = goals_pivot['Колличество должников'] / goals_pivot['Колличество заемщиков'] * 100

In [56]:
# сортируем нашу сводную таблицу по убыванию по столбцу 'Процентное соотношение' при помощи метода sort_values()
goals_pivot.sort_values(by='Процентное соотношение', ascending=False)

,Колличество заемщиков,Колличество должников,Процентное соотношение
goals,,,
Автомобиль,973,90,9.249743
Образование,2699,236,8.743979
Свадьба,793,64,8.070618
Недвижимость,3883,293,7.545712


**Вывод**

Как мы може видеть из данных, основный цели у задолжников это покупка автомобиля и оплата учебы - около 9 %. Наиболее ответственно к выплате кредит подходят там, кто задалс целю купить жилье. Среди них всего 7 % должников.


## Шаг 4. Общий вывод

   В ходе исследования нам далось нарисовать портрет самого нежелательного заемщика - это студент/ка, не обрмененный узами брака, но имеющий детей, который хочет купить машинуу или оплатить учебу. 